In [1]:
#Uninstalling scikit-image, updating dask, and reinstalling scikit-image fixed it.
#conda upgrade ipykernel
#conda install tornado=4.5.3
#conda install -c anaconda toolz

In [2]:
import pymysql
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
import tensorflow as tf

In [3]:
conn = pymysql.connect(user='asaxena6', password='', host='127.0.0.1', database = 'cchs_nov16')

In [4]:
query = "SELECT VisitEventID, VisitIdentifier, PatientIdentifier, EventTimeKey,\
         MinutesFromArrival, HeartRate, RespiratoryRate, Temperature, SystolicBP,\
         WBC, Bands, SedRate, CReactiveProtein, Procalcitonin, DiastolicBP, Platelet,\
         PulseOx, FIO2, MAP, OxygenSource, BUN, Lactate, Creatinine, BiliRubin,\
         CurrentLocationTypeCode from cchs_nov16.Nonshock_Infected order by PatientIdentifier, EventTimeKey;"
query

'SELECT VisitEventID, VisitIdentifier, PatientIdentifier, EventTimeKey,         MinutesFromArrival, HeartRate, RespiratoryRate, Temperature, SystolicBP,         WBC, Bands, SedRate, CReactiveProtein, Procalcitonin, DiastolicBP, Platelet,         PulseOx, FIO2, MAP, OxygenSource, BUN, Lactate, Creatinine, BiliRubin,         CurrentLocationTypeCode from cchs_nov16.Nonshock_Infected order by PatientIdentifier, EventTimeKey;'

In [5]:
pd_shock = pd.read_sql(query, conn)

In [6]:
all_features = ['PatientIdentifier','VisitIdentifier','HeartRate','RespiratoryRate','PulseOx','SystolicBP','Temperature']

In [7]:
pd_shock_rel = pd_shock[all_features]
pd_shock_rel.head()

,PatientIdentifier,VisitIdentifier,HeartRate,RespiratoryRate,PulseOx,SystolicBP,Temperature
0,7,126021,NaN,NaN,NaN,NaN,NaN
1,7,126021,NaN,NaN,NaN,NaN,NaN
2,7,126021,101.0,20.0,97.0,141.0,38.0
3,7,126021,NaN,NaN,NaN,NaN,NaN
4,7,126021,NaN,NaN,NaN,NaN,NaN


In [8]:
pd_shock_rel[pd_shock_rel['PatientIdentifier'] == 3643].head()

,PatientIdentifier,VisitIdentifier,HeartRate,RespiratoryRate,PulseOx,SystolicBP,Temperature
7380,3643,57643,NaN,NaN,NaN,NaN,NaN
7381,3643,57643,NaN,NaN,NaN,NaN,NaN
7382,3643,57643,108.0,34.0,88.0,NaN,NaN
7383,3643,57643,NaN,NaN,NaN,NaN,NaN
7384,3643,57643,NaN,NaN,NaN,NaN,NaN


In [9]:
pd_shock_fill = pd_shock_rel.groupby(['PatientIdentifier','VisitIdentifier']).fillna(method='ffill').fillna(method='bfill')

In [10]:
pd_data = pd.concat([pd_shock_rel[['PatientIdentifier','VisitIdentifier']],pd_shock_fill],axis=1)
pd_data.head()

,PatientIdentifier,VisitIdentifier,HeartRate,RespiratoryRate,PulseOx,SystolicBP,Temperature
0,7,126021,101.0,20.0,97.0,141.0,38.0
1,7,126021,101.0,20.0,97.0,141.0,38.0
2,7,126021,101.0,20.0,97.0,141.0,38.0
3,7,126021,101.0,20.0,97.0,141.0,38.0
4,7,126021,101.0,20.0,97.0,141.0,38.0


In [11]:
#THis patient has data for multiple visits
pd_data[pd_data['PatientIdentifier'] == 3643].head()

,PatientIdentifier,VisitIdentifier,HeartRate,RespiratoryRate,PulseOx,SystolicBP,Temperature
7380,3643,57643,108.0,34.0,88.0,137.0,36.9
7381,3643,57643,108.0,34.0,88.0,137.0,36.9
7382,3643,57643,108.0,34.0,88.0,137.0,36.9
7383,3643,57643,108.0,34.0,88.0,137.0,36.9
7384,3643,57643,108.0,34.0,88.0,137.0,36.9


In [12]:
pd_data_unique = pd_data.drop_duplicates(subset=all_features, keep="first")
pd_data_unique.head()

,PatientIdentifier,VisitIdentifier,HeartRate,RespiratoryRate,PulseOx,SystolicBP,Temperature
0,7,126021,101.0,20.0,97.0,141.0,38.0
5,7,126021,109.0,28.0,94.0,174.0,38.0
7,7,126021,107.0,25.0,99.0,167.0,39.3
10,7,126021,103.0,18.0,98.0,185.0,39.3
12,7,126021,90.0,15.0,99.0,157.0,39.3


In [13]:
pd_patient_group = pd_data_unique.groupby('PatientIdentifier').agg(['count'])
pd_patient_group.head()

,VisitIdentifier,HeartRate,RespiratoryRate,PulseOx,SystolicBP,Temperature
,count,count,count,count,count,count
PatientIdentifier,,,,,,
7,9,9,9,9,9,9
20,47,47,47,47,47,47
29,27,27,27,27,27,27
31,23,23,23,23,23,23
44,56,56,56,56,56,56


In [14]:
#pd_patient_group[pd_patient_group['VisitIdentifier']['count'] == 15]

In [15]:
#pd_patient_group['VisitIdentifier'].nsmallest(20, 'count')

In [16]:
pd_patient_group['VisitIdentifier'].describe()

,count
count,2872.000000
mean,35.986072
std,43.353196
min,2.000000
25%,14.000000
50%,24.000000
75%,41.250000
max,616.000000


In [17]:
len(pd_patient_group[pd_patient_group['VisitIdentifier']['count'] >=30])
#Choosing 30 as the sequence lenght so that we have sufficient data

1095

In [18]:
# Get patients with records greater than 30
pid = pd_patient_group[pd_patient_group['VisitIdentifier']['count'] >=30].index
pid = np.array(pid)
pid

array([    20,     44,    110, ..., 119536, 119726, 119775])

In [19]:
len(pid)

1095

In [20]:
type(pd_patient_group['VisitIdentifier']['count'])

pandas.core.series.Series

In [21]:
pd_data_unique.isnull().values.any()

False

In [22]:
#Number of unique patients
nUniquePatients = len(pid)
nUniquePatients

1095

In [23]:
pd_data_latest_records = pd_data_unique[pd_data_unique['PatientIdentifier'].isin(pid)].groupby('PatientIdentifier').tail(30)
pd_data_latest_records.head()

,PatientIdentifier,VisitIdentifier,HeartRate,RespiratoryRate,PulseOx,SystolicBP,Temperature
53,20,38911,86.0,16.0,94.0,147.0,36.9
54,20,38911,86.0,16.0,94.0,132.0,36.9
55,20,38911,90.0,16.0,94.0,123.0,36.9
56,20,38911,90.0,16.0,94.0,145.0,36.9
57,20,38911,87.0,16.0,94.0,117.0,36.9


In [24]:
#pd_data_latest_records.head()

In [25]:
settings = {
    "data": "eICU_task",
    "num_samples": nUniquePatients,
    "seq_length": 30,
    "num_signals": 5,
    "normalise": True,
    #For conditional GAN
    "cond_dim": 0,
    #For conditional GAN
    "max_val": 1,
    #For conditional GAN - to encode labels in one hot encoding
    "one_hot": None,
    "predict_labels": False,
    "hidden_units_g": 100,
    "hidden_units_d": 100,
    "kappa": 1,
    #Noise Dimensions
    "latent_dim": 10,
#     "learning_rate": 0.1,
#     "batch_size": 28,
    "learning_rate": 0.01,
    "batch_size": 56,
    "num_epochs": 120,
    "D_rounds": 1,
#     "G_rounds": 3,
    "G_rounds": 10,
    "shuffle": True,
    "num_generated_features": 5,
    "batch_mean": False,
    "learn_scale": False,
    "dp": False,
    "batches_per_lot": 1,
    "dp_sigma": 0.6,
    "l2norm_bound": 4,
    "use_time": False,
    "WGAN_clip": False
}
settings

{'D_rounds': 1,
 'G_rounds': 10,
 'WGAN_clip': False,
 'batch_mean': False,
 'batch_size': 56,
 'batches_per_lot': 1,
 'cond_dim': 0,
 'data': 'eICU_task',
 'dp': False,
 'dp_sigma': 0.6,
 'hidden_units_d': 100,
 'hidden_units_g': 100,
 'kappa': 1,
 'l2norm_bound': 4,
 'latent_dim': 10,
 'learn_scale': False,
 'learning_rate': 0.01,
 'max_val': 1,
 'normalise': True,
 'num_epochs': 120,
 'num_generated_features': 5,
 'num_samples': 1095,
 'num_signals': 5,
 'one_hot': None,
 'predict_labels': False,
 'seq_length': 30,
 'shuffle': True,
 'use_time': False}

In [26]:
#60-20-20 Split
#train, validate, test = np.split(pd_data_latest_records.sample(frac=1), [int(.6*len(pd_data_latest_records)), int(.8*len(pd_data_latest_records))])

In [27]:
import math

In [28]:
#80-20 split
train = pd_data_latest_records.iloc[0:30*(math.floor(0.8*(pd_data_latest_records.shape[0]/30)))]
validate = pd_data_latest_records.iloc[30*(math.floor(0.8*(pd_data_latest_records.shape[0]/30))):]

In [29]:
print(train.shape)
print(validate.shape)
train.head()

(26280, 7)
(6570, 7)


,PatientIdentifier,VisitIdentifier,HeartRate,RespiratoryRate,PulseOx,SystolicBP,Temperature
53,20,38911,86.0,16.0,94.0,147.0,36.9
54,20,38911,86.0,16.0,94.0,132.0,36.9
55,20,38911,90.0,16.0,94.0,123.0,36.9
56,20,38911,90.0,16.0,94.0,145.0,36.9
57,20,38911,87.0,16.0,94.0,117.0,36.9


In [30]:
X_train = pd_data_latest_records[['HeartRate','RespiratoryRate','PulseOx','SystolicBP','Temperature']]
X_val = validate[['HeartRate','RespiratoryRate','PulseOx','SystolicBP','Temperature']]
#X_test = test[['HeartRate','RespiratoryRate','PulseOx','SystolicBP','Temperature']]
X_train.head()

,HeartRate,RespiratoryRate,PulseOx,SystolicBP,Temperature
53,86.0,16.0,94.0,147.0,36.9
54,86.0,16.0,94.0,132.0,36.9
55,90.0,16.0,94.0,123.0,36.9
56,90.0,16.0,94.0,145.0,36.9
57,87.0,16.0,94.0,117.0,36.9


In [31]:
settings['seq_length']

30

In [32]:
def reshape_data():
    # labels = {'train': data['Y_train'], 'vali': data['Y_vali'], 'test': data['Y_test']}
    samples = {'train':X_train.values, 'vali':X_val.values}#, 'test':X_test.values}
    # Reshape
    for (k, X) in samples.items():
        samples[k] = X.reshape(-1, settings['seq_length'], settings['num_signals'])
    return samples

In [33]:
def normalise_data(train, vali, low=-1, high=1):
    """ Apply some sort of whitening procedure
    """
    # remember, data is num_samples x seq_length x signals
    # whiten each signal - mean 0, std 1
    mean = np.mean(np.vstack([train, vali]), axis=(0, 1))
    std = np.std(np.vstack([train-mean, vali-mean]), axis=(0, 1))

    normalised_train = (train - mean)/std
    normalised_vali = (vali - mean)/std
    #normalised_test = (test - mean)/std
    return normalised_train, normalised_vali#, normalised_test

In [34]:
# def normalise_data(train, low=-1, high=1):
#     """ Apply some sort of whitening procedure
#     """
#     # remember, data is num_samples x seq_length x signals
#     # whiten each signal - mean 0, std 1
#     #mean = np.mean(np.vstack([train, vali]), axis=(0, 1))
#     #std = np.std(np.vstack([train-mean, vali-mean]), axis=(0, 1))
#     mean = np.mean(train, axis=(0, 1))
#     std = np.std((train-mean), axis=(0, 1))
#     normalised_train = (train - mean)/std
#     #normalised_vali = (vali - mean)/std
#     #normalised_test = (test - mean)/std
#     return normalised_train

In [35]:
#np.vstack([X_train.values,X_val.values]).shape

In [36]:
def de_normalise_data(data,mean,std):
    return (data*std)+mean

In [37]:
def get_samples_and_labels(settings):
    
    samples = reshape_data()
    #train, vali, test = samples['train'], samples['vali'], samples['test']
    train, vali = samples['train'], samples['vali']
    #train_labels, vali_labels, test_labels = labels['train'], labels['vali'], labels['test']
    #assert train_labels.shape[1] == settings['cond_dim']
    # normalise to between -1, 1
    #train, vali, test = normalise_data(train, vali, test)
    train, vali = normalise_data(train, vali)
    #train = normalise_data(train)
    #labels = dict()
    #labels['train'], labels['vali'], labels['test'] = train_labels, vali_labels, test_labels

    samples = dict()
    #samples['train'], samples['vali'], samples['test'] = train, vali, test
    samples['train'], samples['vali'] = train, vali
    

    # futz around with labels
#     # TODO refactor cause this is messy
#     if 'one_hot' in settings and settings['one_hot'] and not settings['data_load_from']:
#         if len(labels['train'].shape) == 1:
#             # ASSUME labels go from 0 to max_val inclusive, find max-val
#             max_val = int(np.max([labels['train'].max(), labels['test'].max(), labels['vali'].max()]))
#             # now we have max_val + 1 dimensions
#             print('Setting cond_dim to', max_val + 1, 'from', settings['cond_dim'])
#             settings['cond_dim'] = max_val + 1
#             print('Setting max_val to 1 from', settings['max_val'])
#             settings['max_val'] = 1

#             labels_oh = dict()
#             for (k, v) in labels.items():
#                 A = np.zeros(shape=(len(v), settings['cond_dim']))
#                 A[np.arange(len(v)), (v).astype(int)] = 1
#                 labels_oh[k] = A
#             labels = labels_oh
#         else:
#             assert settings['max_val'] == 1
#             # this is already one-hot!

#     if 'predict_labels' in settings and settings['predict_labels']:
#         samples, labels = data_utils.make_predict_labels(samples, labels)
#         print('Setting cond_dim to 0 from', settings['cond_dim'])
#         settings['cond_dim'] = 0

    # update the settings dictionary to update erroneous settings
    # (mostly about the sequence length etc. - it gets set by the data!)
    settings['seq_length'] = samples['train'].shape[1]
    settings['num_samples'] = samples['train'].shape[0] + samples['vali'].shape[0]# + samples['test'].shape[0]
    settings['num_signals'] = samples['train'].shape[2]
    settings['num_generated_features'] = samples['train'].shape[2]

    return samples

In [38]:
samples = get_samples_and_labels(settings)

In [39]:
samples['train'].shape

(1095, 30, 5)

In [41]:
#samples['train'][0]

In [42]:
mean = np.mean(np.vstack([X_train.values,X_val.values]), axis=(0, 1))
mean

72.250477422628094

In [43]:
std = np.std(np.vstack([X_train.values - mean,X_val.values - mean]), axis=(0,1))
std

41.739929021365711

## Build Model

In [44]:
# import importlib
# importlib.reload(plotting)

In [45]:
import mod_core_rnn_cell_impl
import mmd
import model
from mmd import rbf_mmd2, median_pairwise_distance, mix_rbf_mmd2_and_ratio
import plotting

from time import time
from math import floor

In [46]:
#Z, X, CG, CD, CS = model.create_placeholders(batch_size, seq_length, latent_dim, num_signals, cond_dim)
batch_size = settings['batch_size']
seq_length = settings['seq_length']
latent_dim = settings['latent_dim']
num_signals = settings['num_signals']
cond_dim = settings['cond_dim']
Z, X = model.create_placeholders(batch_size, seq_length, latent_dim, num_signals, cond_dim)

In [47]:
discriminator_vars = ['hidden_units_d', 'seq_length', 'cond_dim', 'batch_size', 'batch_mean']
discriminator_settings = dict((k, settings[k]) for k in discriminator_vars)
generator_vars = ['hidden_units_g', 'seq_length', 'batch_size', 
                  'num_generated_features', 'cond_dim', 'learn_scale']
generator_settings = dict((k, settings[k]) for k in generator_vars)

In [48]:
CGAN = (cond_dim > 0)
if CGAN: assert not settings['predict_labels']

In [49]:
##Had to install toolz package again to solve error
##ValueError: Function has keyword-only arguments or annotations, use getfullargspec() API which can support them

In [50]:
# D_loss, G_loss = model.GAN_loss(Z, X, generator_settings, discriminator_settings, 
#         kappa, CGAN, CG, CD, CS, wrong_labels=wrong_labels)
D_loss, G_loss = model.GAN_loss(Z, X, generator_settings, discriminator_settings, 
        kappa=None, cond = CGAN, CG=None, CD=None, CS=None, wrong_labels=None)

In [51]:
# D_solver, G_solver, priv_accountant = model.GAN_solvers(D_loss, G_loss, learning_rate, batch_size, 
#         total_examples=samples['train'].shape[0], l2norm_bound=l2norm_bound,
#         batches_per_lot=batches_per_lot, sigma=dp_sigma, dp=dp)
D_solver, G_solver, priv_accountant = model.GAN_solvers(D_loss, G_loss, settings['learning_rate'], batch_size, 
        total_examples=samples['train'].shape[0], l2norm_bound=settings['l2norm_bound'],
        batches_per_lot=settings['batches_per_lot'], sigma=settings['dp_sigma'], dp=settings["dp"])


In [52]:
#G_sample = model.generator(Z, **generator_settings, reuse=True, c=CG)
G_sample = model.generator(Z, **generator_settings, reuse=True, c=None)

In [53]:
# --- evaluation --- #
# frequency to do visualisations
vis_freq = max(14000//settings['num_samples'], 1)
eval_freq = max(7000//settings['num_samples'], 1)

# get heuristic bandwidth for mmd kernel from evaluation samples
heuristic_sigma_training = median_pairwise_distance(samples['vali'])
best_mmd2_so_far = 1000

# optimise sigma using that (that's t-hat)
batch_multiplier = 5000//batch_size
eval_size = batch_multiplier*batch_size
eval_eval_size = int(0.2*eval_size)
eval_real_PH = tf.placeholder(tf.float32, [eval_eval_size, seq_length, settings['num_generated_features']])
eval_sample_PH = tf.placeholder(tf.float32, [eval_eval_size, seq_length, settings['num_generated_features']])
n_sigmas = 2
sigma = tf.get_variable(name='sigma', shape=n_sigmas, initializer=tf.constant_initializer(value=np.power(heuristic_sigma_training, np.linspace(-1, 3, num=n_sigmas))))
mmd2, that = mix_rbf_mmd2_and_ratio(eval_real_PH, eval_sample_PH, sigma)
with tf.variable_scope("SIGMA_optimizer"):
    sigma_solver = tf.train.RMSPropOptimizer(learning_rate=0.05).minimize(-that, var_list=[sigma])
    #sigma_solver = tf.train.AdamOptimizer().minimize(-that, var_list=[sigma])
    #sigma_solver = tf.train.AdagradOptimizer(learning_rate=0.1).minimize(-that, var_list=[sigma])
sigma_opt_iter = 2000
sigma_opt_thresh = 0.001
sigma_opt_vars = [var for var in tf.global_variables() if 'SIGMA_optimizer' in var.name]

In [54]:
# tf.reset_default_graph()
# sess.close()
that

<tf.Tensor 'truediv_12:0' shape=() dtype=float32>

In [55]:
vis_freq
batch_multiplier
eval_freq
eval_size

4984

In [56]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [57]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

vis_Z = model.sample_Z(batch_size, seq_length, latent_dim, settings['use_time'])
if CGAN:
    vis_C = model.sample_C(batch_size, cond_dim, max_val, one_hot)
    if 'mnist' in data:
        if one_hot:
            if cond_dim == 6:
                vis_C[:6] = np.eye(6)
            elif cond_dim == 3:
                vis_C[:3] = np.eye(3)
                vis_C[3:6] = np.eye(3)
            else:
                raise ValueError(cond_dim)
        else:
            if cond_dim == 6:
                vis_C[:6] = np.arange(cond_dim)
            elif cond_dim == 3:
                vis_C = np.tile(np.arange(3), 2)
            else:
                raise ValueError(cond_dim)
    elif 'eICU_task' in data:
        vis_C = labels['train'][np.random.choice(labels['train'].shape[0], batch_size, replace=False), :]
    vis_sample = sess.run(G_sample, feed_dict={Z: vis_Z, CG: vis_C})
else:
    vis_sample = sess.run(G_sample, feed_dict={Z: vis_Z})
    vis_C = None

In [58]:
vis_Z.shape
vis_sample.shape

(56, 30, 5)

In [59]:
num_epochs=settings['num_epochs']
use_time = settings['use_time']
dp = settings['dp']
shuffle = settings['shuffle']
vis_real_indices = np.random.choice(len(samples['vali']), size=6)
vis_real = np.float32(samples['vali'][vis_real_indices, :, :])
#if not labels['vali'] is None:
#    vis_real_labels = labels['vali'][vis_real_indices]
#else:
    #vis_real_labels = None
vis_real_labels = None
identifier = 'LSTMGAN-BS56LR0.01GR10SeqLen30-Non-Sepsis'
plotting.save_plot_sample(vis_real, 0, identifier + '_real', n_samples=6, 
                           num_epochs=settings['num_epochs'])

# for dp
# target_eps = [0.125, 0.25, 0.5, 1, 2, 4, 8]
# dp_trace = open('./experiments/traces/' + identifier + '.dptrace.txt', 'w')
# dp_trace.write('epoch ' + ' eps' .join(map(str, target_eps)) + '\n')

trace = open('./experiments/traces/' + identifier + '.trace.txt', 'w')
trace.write('epoch time D_loss G_loss mmd2 that pdf real_pdf\n')

# --- train --- #
train_vars = ['batch_size', 'D_rounds', 'G_rounds', 'use_time', 'seq_length', 
              'latent_dim', 'num_generated_features', 'cond_dim', 'max_val', 
              'WGAN_clip', 'one_hot']
train_settings = dict((k, settings[k]) for k in train_vars)


t0 = time()
best_epoch = 0
print('epoch\ttime\tD_loss\tG_loss\tmmd2\tthat\tpdf_sample\tpdf_real')
for epoch in range(settings['num_epochs']):
    
    
#     D_loss_curr, G_loss_curr = model.train_epoch(epoch, samples['train'], labels['train'],
#                                         sess, Z, X, CG, CD, CS,
#                                         D_loss, G_loss,
#                                         D_solver, G_solver, 
#                                         **train_settings)

    D_loss_curr, G_loss_curr = model.train_epoch(None, epoch, samples['train'], None,
                                        sess, Z, X, None, None, None,
                                        D_loss, G_loss,
                                        D_solver, G_solver, 
                                        **train_settings)
    # -- eval -- #
    
    # visualise plots of generated samples, with/without labels
    if epoch % vis_freq == 0:
        if CGAN:
            vis_sample = sess.run(G_sample, feed_dict={Z: vis_Z, CG: vis_C})
        else:
            vis_sample = sess.run(G_sample, feed_dict={Z: vis_Z})
        
        one_hot = settings['one_hot']
        predict_labels = settings['predict_labels']
        resample_rate_in_min = None
        multivariate_mnist = None
        plotting.visualise_at_epoch(vis_sample, "", 
                predict_labels, one_hot, epoch, identifier, num_epochs,
                resample_rate_in_min, multivariate_mnist, seq_length, labels=vis_C)
        
    # compute mmd2 and, if available, prob density
    if epoch % eval_freq == 0:
        
        ## how many samples to evaluate with?
        eval_Z = model.sample_Z(eval_size, seq_length, latent_dim, use_time)
#         if 'eICU_task' in data:
#             eval_C = labels['vali'][np.random.choice(labels['vali'].shape[0], eval_size), :]
#         else:
        max_val = None
        eval_C = model.sample_C(eval_size, cond_dim, max_val, one_hot)
        eval_sample = np.empty(shape=(eval_size, seq_length, num_signals))
        for i in range(batch_multiplier):
            if CGAN:
                eval_sample[i*batch_size:(i+1)*batch_size, :, :] = sess.run(G_sample, feed_dict={Z: eval_Z[i*batch_size:(i+1)*batch_size], CG: eval_C[i*batch_size:(i+1)*batch_size]})
            else:
                eval_sample[i*batch_size:(i+1)*batch_size, :, :] = sess.run(G_sample, feed_dict={Z: eval_Z[i*batch_size:(i+1)*batch_size]})
        eval_sample = np.float32(eval_sample)
        eval_real = np.float32(samples['vali'][np.random.choice(len(samples['vali']), size=batch_multiplier*batch_size), :, :])
       
        eval_eval_real = eval_real[:eval_eval_size]
        eval_test_real = eval_real[eval_eval_size:]
        eval_eval_sample = eval_sample[:eval_eval_size]
        eval_test_sample = eval_sample[eval_eval_size:]
        
        ## MMD
        # reset ADAM variables
        sess.run(tf.initialize_variables(sigma_opt_vars))
        sigma_iter = 0
        that_change = sigma_opt_thresh*2
        old_that = 0
        while that_change > sigma_opt_thresh and sigma_iter < sigma_opt_iter:
            new_sigma, that_np, _ = sess.run([sigma, that, sigma_solver], feed_dict={eval_real_PH: eval_eval_real, eval_sample_PH: eval_eval_sample})
            that_change = np.abs(that_np - old_that)
            old_that = that_np
            sigma_iter += 1
        opt_sigma = sess.run(sigma)
        mmd2, that_np = sess.run(mix_rbf_mmd2_and_ratio(eval_test_real, eval_test_sample,biased=False, sigmas=sigma))
        
        ## save parameters
        if mmd2 < best_mmd2_so_far and epoch > 10:
            best_epoch = epoch
            best_mmd2_so_far = mmd2
            model.dump_parameters(identifier + '_' + str(epoch), sess)
        pdf = None
        ## prob density (if available)
        if not pdf is None:
            pdf_sample = np.mean(pdf(eval_sample[:, :, 0]))
            pdf_real = np.mean(pdf(eval_real[:, :, 0]))
        else:
            pdf_sample = 'NA'
            pdf_real = 'NA'
    else:
        # report nothing this epoch
        mmd2 = 'NA'
        #that = 'NA'
        that_np = 'NA'
        pdf_sample = 'NA'
        pdf_real = 'NA'
    
#     ## get 'spent privacy'
#     if settings['dp']:
#         spent_eps_deltas = priv_accountant.get_privacy_spent(sess, target_eps=target_eps)
#         # get the moments
#         deltas = []
#         for (spent_eps, spent_delta) in spent_eps_deltas:
#             deltas.append(spent_delta)
#         dp_trace.write(str(epoch) + ' ' + ' '.join(map(str, deltas)) + '\n')
#         if epoch % 10 == 0: dp_trace.flush()
    
    ## print
    t = time() - t0
    try:
        print('%d\t%.2f\t%.4f\t%.4f\t%.5f\t%.0f\t %s\t %s' % (epoch, t, D_loss_curr, G_loss_curr, mmd2, that_np, pdf_sample, pdf_real))
    except TypeError:       # mmd2 is NA (format as strings)
        print('%d\t%.2f\t%.4f\t%.4f\t%s\t%s\t %s\t %s' % (epoch, t, D_loss_curr, G_loss_curr, mmd2, that_np, pdf_sample, pdf_real))

    ## save trace
    trace.write(' '.join(map(str, [epoch, t, D_loss_curr, G_loss_curr, mmd2, that_np, pdf_sample, pdf_real])) + '\n')
    if epoch % 10 == 0: 
        trace.flush()
        plotting.plot_trace(identifier, xmax=num_epochs, dp=dp)

    if shuffle:     # shuffle the training data 
        perm = np.random.permutation(samples['train'].shape[0])
        samples['train'] = samples['train'][perm]
#         if labels['train'] is not None:
#             labels['train'] = labels['train'][perm]
    
    if epoch % 50 == 0:
        model.dump_parameters(identifier + '_' + str(epoch), sess)

trace.flush()
plotting.plot_trace(identifier, xmax=num_epochs, dp=dp)
model.dump_parameters(identifier + '_' + str(epoch), sess)

## after-the-fact evaluation
#n_test = vali.shape[0]      # using validation set for now TODO
#n_batches_for_test = floor(n_test/batch_size)
#n_test_eval = n_batches_for_test*batch_size
#test_sample = np.empty(shape=(n_test_eval, seq_length, num_signals))
#test_Z = model.sample_Z(n_test_eval, seq_length, latent_dim, use_time)
#for i in range(n_batches_for_test):
#    test_sample[i*batch_size:(i+1)*batch_size, :, :] = sess.run(G_sample, feed_dict={Z: test_Z[i*batch_size:(i+1)*batch_size]})
#test_sample = np.float32(test_sample)
#test_real = np.float32(vali[np.random.choice(n_test, n_test_eval, replace=False), :, :])
## we can only get samples in the size of the batch...
#heuristic_sigma = median_pairwise_distance(test_real, test_sample)
#test_mmd2, that = sess.run(mix_rbf_mmd2_and_ratio(test_real, test_sample, sigmas=heuristic_sigma, biased=False))
##print(test_mmd2, that)

epoch	time	D_loss	G_loss	mmd2	that	pdf_sample	pdf_real
0	72.00	1.2250	0.5275	1.00528	7986	 NA	 NA


/home/asaxena6/plotting.py:288: FutureWarning: read_table is deprecated, use read_csv instead.
  da = read_table(trace_path, sep=' ')


Recorded 9 parameters to ./experiments/parameters/LSTMGAN-BS56LR0.01GR10SeqLen30-Non-Sepsis_0.npy
1	103.82	0.3226	2.8688	NA	NA	 NA	 NA
2	138.13	0.5340	2.3951	NA	NA	 NA	 NA
3	170.47	0.2350	3.0397	NA	NA	 NA	 NA
4	201.10	0.4144	2.6314	NA	NA	 NA	 NA
5	258.22	3.8541	0.0818	0.58882	262	 NA	 NA
6	287.09	1.5615	0.6195	NA	NA	 NA	 NA
7	315.81	1.0333	1.2779	NA	NA	 NA	 NA
8	344.61	1.4102	0.6524	NA	NA	 NA	 NA
9	372.95	1.5714	0.5470	NA	NA	 NA	 NA
10	427.85	1.3059	0.8715	0.39450	187	 NA	 NA
11	456.68	1.7312	0.4628	NA	NA	 NA	 NA
12	484.54	1.6159	0.5346	NA	NA	 NA	 NA
13	511.01	0.9562	1.3652	NA	NA	 NA	 NA
14	539.02	0.7282	1.9552	NA	NA	 NA	 NA
Recorded 9 parameters to ./experiments/parameters/LSTMGAN-BS56LR0.01GR10SeqLen30-Non-Sepsis_15.npy
15	588.76	1.6572	0.5188	0.11192	117	 NA	 NA
16	616.13	1.3711	0.6965	NA	NA	 NA	 NA
17	646.55	1.1513	1.0442	NA	NA	 NA	 NA
18	673.34	0.9367	1.3271	NA	NA	 NA	 NA
19	699.54	1.3397	0.7835	NA	NA	 NA	 NA
20	757.44	1.4413	0.6770	0.18201	120	 NA	 NA
21	791.37	1.3731	0.7194	NA	N

True

In [60]:
print(samples['train'].shape)
print(samples['vali'].shape)

(1095, 30, 5)
(219, 30, 5)


In [61]:
real_data = np.vstack((samples['train'],samples['vali']))
np.save('./data/'+identifier+'real_data',real_data)

In [62]:
real_data = np.load('./data/'+identifier+'real_data.npy')

In [63]:
real_data.shape

(1314, 30, 5)

In [25]:
#real_data[0]

In [65]:
deNorm_real_data = de_normalise_data(real_data,mean,std)
deNorm_real_data.shape

(1314, 30, 5)

In [66]:
#deNorm_real_data[0]

In [67]:
np.save('./data/'+identifier+'deNorm_real_data',deNorm_real_data)

In [68]:
deNorm_real_data = np.load('./data/'+identifier+'deNorm_real_data.npy')
deNorm_real_data.shape

(1314, 30, 5)

In [69]:
#75-25 train test split
n_samples_fake = samples['train'].shape[0]
n_batches_for_fake = floor(n_samples_fake/settings['batch_size'])
n_test_eval = n_batches_for_fake*settings['batch_size']
test_sample = np.empty(shape=(n_test_eval, seq_length, num_signals))
for i in range(n_batches_for_fake):
    fake_Z = model.sample_Z(settings['batch_size'], seq_length, latent_dim, use_time)
    test_sample[i*settings['batch_size']:(i+1)*settings['batch_size'], :, :] = sess.run(G_sample, feed_dict={Z: fake_Z})

In [70]:
fake_data = test_sample
fake_data.shape

(1064, 30, 5)

In [71]:
np.save('./data/'+identifier+'fake_data',fake_data)

In [72]:
fake_data = np.load('./data/'+identifier+'fake_data.npy')
fake_data.shape

(1064, 30, 5)

In [73]:
deNorm_fake_data = de_normalise_data(fake_data,mean,std)
deNorm_fake_data.shape

(1064, 30, 5)

In [74]:
#deNorm_fake_data[0]

In [75]:
np.save('./data/'+identifier+'deNorm_fake_data',deNorm_fake_data)

In [76]:
deNorm_fake_data = np.load('./data/'+identifier+'deNorm_fake_data.npy')
deNorm_fake_data.shape

(1064, 30, 5)

In [77]:
#model.generator.save(identifier+'.h5')

In [78]:
#Generate Data

In [79]:
# eval_Z = model.sample_Z(56, seq_length, latent_dim, use_time)

In [80]:
# eval_Z.shape

In [ ]:
# eval_sample = sess.run(G_sample, feed_dict={Z: eval_Z})
# eval_sample = np.float32(eval_sample)

In [ ]:
# eval_sample[0]

In [ ]:
# nvidia-smi

In [ ]:
# clip_ops = []
# for var in discriminator_vars:
#     clip_bounds = [-.01, .01]
#     clip_ops.append(
#         tf.assign(var, tf.clip_by_value(var, clip_bounds[0], clip_bounds[1]))
#     )
# clip_disc_weights = tf.group(*clip_ops)

In [ ]:
# clip_disc_weights